# So you Want to Calculate an Impact Risk

Here we walk you through the steps necesssary to calculate the impact risk for a specified orbit, in this case 2024 YR4.
We're going to take a few steps:

1. Fetch the state vector for 2024 YR4 from JPL Horizons
2. Run calculate impacts, it will create and propagate samples through time and check for collisions.
3. Summarize the results into an impact risk.


Before we can start, you will need to install a couple packages.

```bash
pip install adam-core adam-assist
```


In [13]:
# Fetch the state vector from Small Bodies Database (SBDB)
from adam_core.orbits.query import query_sbdb

yr4 = query_sbdb(["2024 YR4"])

print(yr4.to_dataframe())

  orbit_id   object_id  coordinates.x  coordinates.y  coordinates.z  \
0    00000  (2024 YR4)      -0.554305       1.039765      -0.031525   

   coordinates.vx  coordinates.vy  coordinates.vz  coordinates.time.days  \
0       -0.019574       -0.000133       -0.001166                  60693   

   coordinates.time.nanos                      coordinates.covariance.values  \
0                       0  [2.6790681662875186e-12, -4.746648923989143e-1...   

  coordinates.origin.code  
0                     SUN  


In [14]:
# We need to know how many days out to run the propagation,
# relative to when the orbit state vector is defined.
# Let's go 30 days past the possible 2032 impact of 2024 YR4
from adam_core.time import Timestamp
approx_impact_date = Timestamp.from_iso8601(["2032-12-22"], scale="tdb")
thirty_days_after_impact = approx_impact_date.add_days(30)
days_until_thirty_days_after_impact, _ = thirty_days_after_impact.difference(yr4.coordinates.time)
print(days_until_thirty_days_after_impact)

[
  2925
]


In [15]:
# Now we initialize our propagator, and pass it to our calculate_impacts function
from adam_core.dynamics.impacts import calculate_impacts
from adam_assist import ASSISTPropagator

propagator = ASSISTPropagator()

final_states, earth_impacts = calculate_impacts(
    yr4,
    days_until_thirty_days_after_impact[0].as_py(),
    propagator,
    num_samples=1000,
    processes=10, # Multiprocessing speeds things up if you have the CPUs
)

print(earth_impacts)

EarthImpacts(size=19)


In [16]:
# Now we can summarize the returns, a simple ratio in our case
# with only 1 orbit considered
from adam_core.dynamics.impacts import calculate_impact_probabilities
ip = calculate_impact_probabilities(final_states, earth_impacts)
print(ip.to_dataframe())

  orbit_id  impacts  variants  cumulative_probability  mean_impact_time.days  \
0    00000       19      1000                   0.019                  63588   

   mean_impact_time.nanos  stddev_impact_time  minimum_impact_time.days  \
0          50517116965167             0.00501                     63588   

   minimum_impact_time.nanos  maximum_impact_time.days  \
0             49983530294895                     63588   

   maximum_impact_time.nanos  
0             51346236683428  


And there you have it. 